In [1]:
import mat73
data_dict1 = mat73.loadmat('/Users/yeseullee/Documents/ECE271B/data/data/Part_1.mat')
data_dict2 = mat73.loadmat('/Users/yeseullee/Documents/ECE271B/data/data/Part_2.mat')
data_dict3 = mat73.loadmat('/Users/yeseullee/Documents/ECE271B/data/data/Part_3.mat')
data_dict4 = mat73.loadmat('/Users/yeseullee/Documents/ECE271B/data/data/Part_4.mat')


In [2]:
import numpy as np
import os
from scipy.signal import find_peaks
import pandas as pd
import matplotlib.pyplot

dataset_path=['/Users/yeseullee/Documents/ECE271B/MIMICii/part1',
'/Users/yeseullee/Documents/ECE271B/MIMICii/part2',
'/Users/yeseullee/Documents/ECE271B/MIMICii/part3',
'/Users/yeseullee/Documents/ECE271B/MIMICii/part4']
data_dicts=[data_dict1['Part_1'],data_dict2['Part_2'],data_dict3['Part_3'],data_dict4['Part_4']]

freq=125
sec=5

label_df=pd.DataFrame({'subject_id':[],'hypertension_level':[]})

for part_num,data_dict in enumerate(data_dicts):
    for sub_num,sub in enumerate(data_dict):
        file_name1=str(sub_num+1)
        
        #sub[0]=>PPG
        seg_cnt=0
        for i in range(0,len(sub[0]),freq*sec):
            if i+(freq*sec)>len(sub[0]): #remove last segment(due to length)
                break
            seg_cnt+=1
            file_name2='_'+str(seg_cnt)
            file_name=file_name1+file_name2
            save_ppg=np.round(sub[0][i:i+(freq*sec)],4)

            #peak detection on ABP to get Hypertension label
            peak_idx, peak_val=find_peaks(sub[1][i:i+(freq*sec)],height=60)
            if len(peak_idx)==0: #if there is no peak in ABP
                print('Part: ',part_num+1)
                print(file_name)
                continue

            if peak_val['peak_heights'].max()<=120:
                label_df=label_df.append({'subject_id':file_name,'hypertension_level':'Normal'},ignore_index=True)
            elif peak_val['peak_heights'].max()>120 and peak_val['peak_heights'].max()<=139:
                label_df=label_df.append({'subject_id':file_name,'hypertension_level':'Prehypertension'},ignore_index=True)
            elif peak_val['peak_heights'].max()>139:
                label_df=label_df.append({'subject_id':file_name,'hypertension_level':'Hypertension'},ignore_index=True)

            np.savetxt(os.path.join(dataset_path[part_num],file_name+'.txt'),save_ppg,newline='\t')

    label_df.to_csv(os.path.join(dataset_path[part_num],'label'+str(part_num+1)+'.csv'),sep=',',index=False)


Part:  1
1614_1
Part:  2
573_26
Part:  3
253_14
